Questo Notebook contiene un classificatore stupido che classifica tutti i record con lo stato che avevano all’istante precedente.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
from scipy.ndimage.interpolation import shift
from math import sin, cos, sqrt, atan2, radians 
from sklearn import tree, svm, linear_model, ensemble, neighbors, naive_bayes 
import dateutil
import os
from pathlib import Path
from joblib import dump, load
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import make_scorer
import ast


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_path = Path('../filtered_status.csv')
stations_path = Path('../station.csv')

In [3]:
status_df = pd.read_csv(status_path, parse_dates=['time'])
stations_df = pd.read_csv(stations_path)

In [4]:
status_df

,station_id,bikes_available,docks_available,time
0,2,2,25,2013-08-29 12:06:01
1,2,2,25,2013-08-29 12:07:01
2,2,2,25,2013-08-29 12:08:01
3,2,2,25,2013-08-29 12:09:01
4,2,2,25,2013-08-29 12:10:01
...,...,...,...,...
71977905,84,8,7,2015-08-31 23:55:02
71977906,84,8,7,2015-08-31 23:56:01
71977907,84,8,7,2015-08-31 23:57:02
71977908,84,8,7,2015-08-31 23:58:02


In [5]:
stations_df

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


In [4]:
#il seguente vettore è stato ottenuto in analisi fatte precedentemente. 
SFancisco_stations = [39, 41, 42, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 59, 60, 
                      61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 82]

### Calcolo la distanza tra le diverse stazioni della città di San Francisco

In [5]:
#function to retrieve distance between 2 stations
def getDistance(lat_a, long_a, lat_b, long_b):
    # approximate radius of earth in km
    R = 6373.0    
    lat1=radians(lat_a)
    lat2=radians(lat_b)
    lon1=radians(long_a)
    lon2=radians(long_b)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [6]:
coordinates_stations_df = stations_df[stations_df["id"].isin(SFancisco_stations)]
coordinates_stations_df = coordinates_stations_df[["id", "lat", "long"]]
records = coordinates_stations_df.to_records(index=False)
list_coo = list(records)
# print(list_coo)

voc_distances={}
for i in range(len(list_coo)):
    for j in range(i+1,len(list_coo)):
        station1=list_coo[i][0]
        station2=list_coo[j][0]
        lat_i=float(list_coo[i][1])
        long_i=float(list_coo[i][2])
        lat_j=float(list_coo[j][1])
        long_j=float(list_coo[j][2])
        distance=getDistance(lat_i, long_i, lat_j, long_j)
        id_stations=str(station1)+' '+str(station2)
        voc_distances[id_stations]=distance


In [16]:
# voc_distances

In [7]:
distance_dictionary = {}


for station_id in SFancisco_stations:
    distance_dictionary[station_id] = []
    
    delta0_stations = []
    delta1_stations = []
    delta2_stations = []

    for key in voc_distances.keys():
        if str(station_id) in key:
            distance = voc_distances[key]

            if key.split(" ")[0] == str(station_id):
                other=key.split(" ")[1]
            elif key.split(" ")[1] == str(station_id):
                other=key.split(" ")[0]
            
            if distance<=1:
                delta0_stations.append(other)         
            elif distance<=2:
                delta1_stations.append(other)
            elif distance<=3:
                delta2_stations.append(other)
        
    distance_dictionary[station_id].append(list(delta0_stations))
    distance_dictionary[station_id].append(list(delta1_stations))
    distance_dictionary[station_id].append(list(delta2_stations))


In [ ]:
#Questo dizionario contiene come chiave tutte le stazioni e come valori corrisopondenti tre liste contenenti
# le stazioni che distantano dalla stazione della chiave rispettivamente <1 km, tra 1 e  

# distance_dictionary

In [8]:
# distance_dictionary[63]

### Classificazione

In [3]:
#Parametri considerati per la costruzione dei dataset della classificazione
time_interval = 30
window_width = 5

#Parametri usati per l'estazione dei pattern
space_interval = 1000

#Parametri usati per il filtraggio dei pattern
conf_threshold = 40 #valore intero (%)
sup_threshold = 0

#Flag utile a considerare soltanto i pattern contenenti esclusivamente QuasiPiena
filter_normal = True

#Numero di pattern da metchare (in questo caso non servono)
# num_tot_pat = 1

#Lista di fasce temporali da considerare
time_slots = [
    "0-6", 
    "6-10", 
    "10-14", 
    "14-17", 
    "17-20", 
    "20-24"
]

In [17]:
for slot in time_slots:
    print(f"---- TIME SLOT {slot} ----\n")
    #Se considero soltanto i pattern contenenti soltanto QuasiPiena
    if filter_normal:
        pattern_path = f"./Results_extraction/{time_interval}min_{space_interval}m/Time_slots/filtered_results_{slot}_({conf_threshold}%-{sup_threshold})_StateChange_almostFull_{time_interval}_{space_interval}_0(3-3)_ordered_by_confidence.txt"
        output_path = f'Test_results/{time_interval}min/Time_slots/results_{slot}_Dumb_classifier_almostFull_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt'
    else:
        pattern_path = f"./Results_extraction/{time_interval}min_{space_interval}m/Time_slots/filtered_results_{slot}_({conf_threshold}%-{sup_threshold})_StateChange_Normal_almostFull_{time_interval}_{space_interval}_0(3-3)_ordered_by_confidence.txt"
        output_path = f'Test_results/{time_interval}min/Time_slots/results_{slot}_Dumb_classifier_Normal_almostFull_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt'

    file = open(output_path, "w")
    file.write(f'PATTERN TESING RESULTS:\n\n')
    tot_fp = 0
    tot_tp = 0
    tot_fn = 0
    tot_tn = 0


    for station_id in SFancisco_stations:

        test_path = Path(f'../Traditional_classification/datasets/{time_interval}_{window_width}/Train_test_with_time/station{station_id}_test.csv')
        test_df = pd.read_csv(test_path)
        
        time_slot_test_df = test_df.set_index("time", drop=True)
        time_slot_test_df.index=pd.to_datetime(time_slot_test_df.index)
        start_time= slot.split('-')[0]
        end_time = slot.split('-')[1]
        if end_time != '24':       
            time_slot_test_df = time_slot_test_df.between_time(f'{start_time}:00', f'{end_time}:00')
        else:
            time_slot_test_df = time_slot_test_df.between_time(f'{start_time}:00', '23:59')

        y_test = time_slot_test_df["status"]
        # per considerare come predizione lo stato nell'istante precedente slotto in avanti la y_true di 1
        y_pred = y_test.shift(periods =+1)
        
        # elimino il primo elemento di entrambi in quanto in y_pred sarebbe NaN
        y_test.drop(index=y_test.index[0], axis=0, inplace=True)
        y_pred.drop(index=y_pred.index[0], axis=0, inplace=True)

        cm = confusion_matrix(y_test, y_pred, labels=["N", "QP"])

        str_= f'PATTERN TEST FOR STATION {station_id}' + '\n'
        str_ += f'Confusion matrix:' + '\n'
        str_ += str(cm) + '\n'

        tn, fp, fn, tp = cm.ravel()
        str_+= f'tp={tp}, fn={fn}, fp={fp}, tn={tn}' +'\n'

        test_accuracy = (tn + tp) / (tn + fp + fn + tp)
        #test_recall = (tp) / (tp + fn)
        #test_precision = (tp) / (tp + fp)
        test_recall = recall_score(y_test, y_pred, pos_label='QP', zero_division=0)
        test_precision = precision_score(y_test, y_pred, pos_label='QP', zero_division=0)
        test_f1_score = f1_score(y_test, y_pred, pos_label='QP', zero_division=0)

        str_+= f'accuracy={test_accuracy}; recall={test_recall}; precision={test_precision}; f1_score= {test_f1_score}' +'\n\n'
        str_+= "-"*10 +'\n\n'

        tot_fp += fp
        tot_tp += tp
        tot_fn += fn
        tot_tn += tn

        file.write(str_)
#         print(str_)

    avg_accuracy = (tot_tn + tot_tp) / (tot_tn + tot_fp + tot_fn + tot_tp)
    avg_recall = (tot_tp) / (tot_tp + tot_fn)
    if (tot_tp + tot_fp) != 0.0:
        avg_precision = (tot_tp) / (tot_tp + tot_fp)
    else:
        avg_precision = 0.0
    
    if avg_recall!= 0.0 and avg_precision!= 0.0:
        avg_f1_score = 2*(1/((1/avg_recall)+(1/avg_precision)))
    else:
        avg_f1_score = 0.0

    avg_str = "AVERAGE VALUES FOR PATTERN TEST:\n"
    avg_str += "Confusion matrix:\n"
    avg_str += f"[[{tot_tn} {tot_fp}]\n[{tot_fn} {tot_tp}]]\n"
    avg_str += f"tot_tp={tot_tp}, tot_fn={tot_fn}, tot_fp={tot_fp}, tot_tn={tot_tn}\n"
    avg_str += f"accuracy={avg_accuracy}; recall={avg_recall}; precision={avg_precision}; f1_score={avg_f1_score}\n\n"
    print(avg_str)
    file.write(avg_str)
    file.close()


---- TIME SLOT 0-6 ----

AVERAGE VALUES FOR PATTERN TEST:
Confusion matrix:
[[73511 401]
[399 3214]]
tot_tp=3214, tot_fn=399, tot_fp=401, tot_tn=73511
accuracy=0.9896807481457595; recall=0.8895654580680875; precision=0.889073305670816; f1_score=0.8893193137797454


---- TIME SLOT 6-10 ----

AVERAGE VALUES FOR PATTERN TEST:
Confusion matrix:
[[50951 851]
[856 822]]
tot_tp=822, tot_fn=856, tot_fp=851, tot_tn=50951
accuracy=0.968081525804039; recall=0.4898688915375447; precision=0.4913329348475792; f1_score=0.49059982094897037


---- TIME SLOT 10-14 ----

AVERAGE VALUES FOR PATTERN TEST:
Confusion matrix:
[[51533 610]
[611 971]]
tot_tp=971, tot_fn=611, tot_fp=610, tot_tn=51533
accuracy=0.9772731503024663; recall=0.6137800252844501; precision=0.614168247944339; f1_score=0.6139740752450206


---- TIME SLOT 14-17 ----

AVERAGE VALUES FOR PATTERN TEST:
Confusion matrix:
[[40045 525]
[524 486]]
tot_tp=486, tot_fn=524, tot_fp=525, tot_tn=40045
accuracy=0.9747715247715247; recall=0.4811881188118

#### Tabella contenente tutti i risultati considerando un intervallo di 30 minuti e come stato soltanto Quasi Piena

Script che va a leggere i risultati finali e li salva in un dataframe in modo da visualizzarli tutti insieme

In [5]:
rows = []
result_matrix = []
columns = ["avg_accuracy", "avg_recall", "avg_precision", "avg_f1_score"]

for slot in time_slots:
    input_file_name = f"results_{slot}_Dumb_classifier_almostFull_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt"
    input_file_path = f"Test_results/{time_interval}min/Time_slots/{input_file_name}"
    
    row = f"time_slot_{slot}"
    rows.append(row)
    with open(input_file_path, "r") as file:
        lines = file.read().splitlines()
        last_line = lines[-2]
#             print (last_line)
        results = last_line.split(';')
        result_values = []
        for result in results:
#                 print (result)
            value = round(float(result.split('=')[1]), 4)
            result_values.append(value)
        result_matrix.append(result_values)

results_table = pd.DataFrame(result_matrix, columns=columns, index=pd.Index(rows))
# print(results_table.to_latex(float_format="%.4f", bold_rows=True))
results_table = results_table.drop(columns=["avg_accuracy"])
print(results_table.to_latex(bold_rows=True))
results_table

\begin{tabular}{lrrr}
\toprule
{} &  avg\_recall &  avg\_precision &  avg\_f1\_score \\
\midrule
\textbf{time\_slot\_0-6  } &      0.8896 &         0.8891 &        0.8893 \\
\textbf{time\_slot\_6-10 } &      0.4899 &         0.4913 &        0.4906 \\
\textbf{time\_slot\_10-14} &      0.6138 &         0.6142 &        0.6140 \\
\textbf{time\_slot\_14-17} &      0.4812 &         0.4807 &        0.4810 \\
\textbf{time\_slot\_17-20} &      0.5203 &         0.5203 &        0.5203 \\
\textbf{time\_slot\_20-24} &      0.7848 &         0.7852 &        0.7850 \\
\bottomrule
\end{tabular}



,avg_recall,avg_precision,avg_f1_score
time_slot_0-6,0.8896,0.8891,0.8893
time_slot_6-10,0.4899,0.4913,0.4906
time_slot_10-14,0.6138,0.6142,0.6140
time_slot_14-17,0.4812,0.4807,0.4810
time_slot_17-20,0.5203,0.5203,0.5203
time_slot_20-24,0.7848,0.7852,0.7850


#### Tabella contenente tutti i risultati considerando un intervallo di 30 minuti e come stati Normal e Quasi Piena

In [19]:
time_slots_dict ={
    "0-6" : [1],     # 287 pattern totali (40%)
    "6-10" : [1],    # 159 pattern totali (40%) 
    "10-14" : [1],     # 306 pattern totali (40%)
    "14-17" : [1],     # 287 pattern totali (40%)
    "17-20" : [1],     # 323 pattern totali (40%)
    "20-24" : [1]    # 301 pattern totali (40%)
}

best_results ={
    "0-6" : [18],     # 22 pattern totali
    "6-10" : [15],    # 20 pattern totali
    "10-14" : [25],     # 41 pattern totali
    "14-17" : [25],     # 32 pattern totali
    "17-20" : [30],     # 45 pattern totali
    "20-24" : [15]    # 21 pattern totali
}

current_conf_threshold = 55
num_pattern_list = [1] # selezionare il numero di patern desiderato
rows = []
result_matrix = []
columns = ["accuracy", "recall", "precision", "f1_score"]

for slot in time_slots_dict.keys():
    for num_pat in num_pattern_list:
#     for num_pat in best_results[slot]:
        if filter_normal:
            status_str= "QP"
            input_file_name = f"results_{slot}_almostFull_{num_pat}match_{time_interval}min_{window_width}_({current_conf_threshold}%-{sup_threshold}).txt"
        else:
            status_str= "N/QP"
            input_file_name = f"results_{slot}_Normal_almostFull_{num_pat}match_{time_interval}min_{window_width}_({current_conf_threshold}%-{sup_threshold}).txt"
        
        input_file_path = f"Test_results/{time_interval}min/Time_slots/{input_file_name}"

        row = f"{status_str}_conf{conf_threshold}%_{slot}_{num_pat}match"
        rows.append(row)
        with open(input_file_path, "r") as file:
            lines = file.read().splitlines()
            last_line = lines[-2]
#             print (last_line)
            results = last_line.split(';')
            result_values = []
            for result in results:
#                 print (result)
                value = round(float(result.split('=')[1]), 4)
                result_values.append(value)
            result_matrix.append(result_values)

results_table = pd.DataFrame(result_matrix, columns=columns, index=pd.Index(rows))
# print(results_table.to_latex(float_format="%.4f", bold_rows=True))
results_table

,accuracy,recall,precision,f1_score
N/QP_conf55%_0-6_1match,0.9925,0.9358,0.9065,0.9209
N/QP_conf55%_6-10_1match,0.9686,0.0000,0.0000,0.0000
N/QP_conf55%_10-14_1match,0.9688,0.4520,0.4695,0.4606
N/QP_conf55%_14-17_1match,0.9757,0.0000,0.0000,0.0000
N/QP_conf55%_17-20_1match,0.9604,0.3373,0.3780,0.3565
N/QP_conf55%_20-24_1match,0.9800,0.6984,0.7622,0.7289


#### Valuto le prestazioni generali del modello, considerando alcuni valori di default per il numero minimo di pattern. In questo modo possiamo vedere come si comporta generalmente il modello.

Vado quindi a ricavare e sommare le varie matrici di confusione iterando sulle diverse fasce orarie, per i diversi valori di default per il numero minimo di pattern.

In [32]:
total_conf_matrix = {}
result_matrix = []
default_num_pat =[15, 18, 20, 25, 30] 

for num_pat in default_num_pat: 
    print(f"NUMERO MINIMO DI PATTERN:{num_pat}")
    all_fp=0
    all_tp=0
    all_fn=0
    all_tn=0
        
    for slot in time_slots:
        
        input_file_name = f"results_{slot}_almostFull_{num_pat}match_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt"
        input_file_path = f"./Test_results/{time_interval}min/Time_slots/{input_file_name}" 
        
        with open(input_file_path, "r") as file:
            lines = file.read().splitlines()
            last_line = lines[-3]
#             print (last_line)
            results = last_line.split(', ')
            result_values = []
            for result in results:
#                 print (result)
                value = int(result.split('=')[1])
                result_values.append(value)
        all_tp+=result_values[0]   
        all_fn+=result_values[1]
        all_fp+=result_values[2]
        all_tn+=result_values[3]
        
    avg_accuracy = (all_tn + all_tp) / (all_tn + all_fp + all_fn + all_tp)
    avg_recall = (all_tp) / (all_tp + all_fn)
    if (all_tp + all_fp) != 0.0:
        avg_precision = (all_tp) / (all_tp + all_fp)
    else:
        avg_precision = 0.0

    if avg_recall!= 0.0 and avg_precision!= 0.0:
        avg_f1_score = 2*(1/((1/avg_recall)+(1/avg_precision)))
    else:
        avg_f1_score = 0.0

    result_matrix.append([avg_accuracy, avg_recall, avg_precision, avg_f1_score])
    final_str = f"[[{all_tn} {all_fp}]\n[{all_fn} {all_tp}]]\n"
    final_str += f"all_tp={all_tp}, all_fn={all_fn}, all_fp={all_fp}, all_tn={all_tn}\n"
    final_str += f"accuracy={round(avg_accuracy, 5)}; recall={round(avg_recall, 5)}; precision={round(avg_precision, 5)}; f1_score={round(avg_f1_score, 5)}\n\n"
    print (final_str)
    total_conf_matrix[num_pat] = [all_tp, all_fn, all_fp, all_tn]

result_table = pd.DataFrame(result_matrix, columns=['avg_accuracy', 'avg_recall', 'avg_precision', 'avg_f1_score'], index=pd.Index(default_num_pat))
result_table.to_csv(f"./Test_results/{time_interval}min/Time_slots/Overall_results_AllSlots.csv")
result_table

NUMERO MINIMO DI PATTERN:15
[[304740 157]
[10759 324]]
all_tp=324, all_fn=10759, all_fp=157, all_tn=304740
accuracy=0.96545; recall=0.02923; precision=0.6736; f1_score=0.05604


NUMERO MINIMO DI PATTERN:18
[[304824 73]
[10927 156]]
all_tp=156, all_fn=10927, all_fp=73, all_tn=304824
accuracy=0.96519; recall=0.01408; precision=0.68122; f1_score=0.02758


NUMERO MINIMO DI PATTERN:20
[[304855 42]
[11008 75]]
all_tp=75, all_fn=11008, all_fp=42, all_tn=304855
accuracy=0.96503; recall=0.00677; precision=0.64103; f1_score=0.01339


NUMERO MINIMO DI PATTERN:25
[[304887 10]
[11056 27]]
all_tp=27, all_fn=11056, all_fp=10, all_tn=304887
accuracy=0.96498; recall=0.00244; precision=0.72973; f1_score=0.00486


NUMERO MINIMO DI PATTERN:30
[[304894 3]
[11073 10]]
all_tp=10, all_fn=11073, all_fp=3, all_tn=304894
accuracy=0.96495; recall=0.0009; precision=0.76923; f1_score=0.0018




,avg_accuracy,avg_recall,avg_precision,avg_f1_score
15,0.965454,0.029234,0.673597,0.056036
18,0.965188,0.014076,0.681223,0.027581
20,0.965029,0.006767,0.641026,0.013393
25,0.964979,0.002436,0.729730,0.004856
30,0.964947,0.000902,0.769231,0.001802
